In [1]:
#r "nuget: Yueyinqiu.Su.D2lTorchSharp"

Installed Packages Yueyinqiu.Su.D2lTorchSharp, 0.0.3

Loading extensions from `C:\Users\yueyinqiu\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

2.1.1. 入门

In [2]:
using torch = TorchSharp.torch;

In [3]:
var x = torch.arange(12);
x

[12], type = Int64, device = cpu
 0 1 2 3 4 5 6 7 8 9 10 11

In [4]:
x.shape

[ 12 ]

In [5]:
x.numel()

12

In [6]:
var X = x.reshape(3, 4);
X

[3x4], type = Int64, device = cpu
 0 1 2 3
 4 5 6 7
 8 9 10 11

In [7]:
torch.zeros([2, 3, 4])

[2x3x4], type = Float32, device = cpu
[0,..,..] =
 0 0 0 0
 0 0 0 0
 0 0 0 0

[1,..,..] =
 0 0 0 0
 0 0 0 0
 0 0 0 0

In [8]:
torch.ones([2, 3, 4])

[2x3x4], type = Float32, device = cpu
[0,..,..] =
 1 1 1 1
 1 1 1 1
 1 1 1 1

[1,..,..] =
 1 1 1 1
 1 1 1 1
 1 1 1 1

In [9]:
torch.randn(3, 4)

[3x4], type = Float32, device = cpu
 0.31395 -1.6062 -0.50749 -0.52655
 0.39413 -0.10017 0.5017 1.3473
 1.3378 2.0871 -0.331 0.2032

In [10]:
torch.tensor(new long[,] {{ 2, 1, 4, 3 }, { 1, 2, 3, 4 }, { 4, 3, 2, 1 }})

[3x4], type = Int64, device = cpu
 2 1 4 3
 1 2 3 4
 4 3 2 1

2.1.2. 运算符

In [11]:
var x = torch.tensor(new double[] { 1.0, 2, 4, 8 });
var y = torch.tensor(new long[] { 2, 2, 2, 2 });
(x + y, x - y, x * y, x / y, torch.pow(x, y))

Item1,"[4], type = Float64, device = cpu 3 4 6 10"
Item2,"[4], type = Float64, device = cpu -1 0 2 6"
Item3,"[4], type = Float64, device = cpu 2 4 8 16"
Item4,"[4], type = Float64, device = cpu 0.5 1 2 4"
Item5,"[4], type = Float64, device = cpu 1 4 16 64"


In [12]:
torch.exp(x)

[4], type = Float64, device = cpu
 2.7183 7.3891 54.598 2981

In [13]:
var X = torch.arange(12, dtype: torch.float32).reshape([3, 4]);
var Y = torch.tensor(new double[,] {{ 2.0, 1, 4, 3 }, { 1, 2, 3, 4 }, { 4, 3, 2, 1 }});
(torch.cat([X, Y], dim: 0), torch.cat([X, Y], dim: 1))

Item1,"[6x4], type = Float64, device = cpu 0 1 2 3 4 5 6 7 8 9 10 11 2 1 4 3 1 2 3 4 4 3 2 1"
Item2,"[3x8], type = Float64, device = cpu 0 1 2 3 2 1 4 3 4 5 6 7 1 2 3 4 8 9 10 11 4 3 2 1"


In [14]:
X == Y

[3x4], type = Bool, device = cpu
 False True False True
 False False False False
 False False False False

In [15]:
X.sum()

[], type = Float32, device = cpu, value = 66

2.1.3. 广播机制

In [16]:
var a = torch.arange(3).reshape([3, 1]);
var b = torch.arange(2).reshape([1, 2]);
(a, b)

Item1,"[3x1], type = Int64, device = cpu 0 1 2"
Item2,"[1x2], type = Int64, device = cpu 0 1"


In [17]:
a + b

[3x2], type = Int64, device = cpu
 0 1
 1 2
 2 3

2.1.4. 索引和切片

In [18]:
(X[-1], X[1..3])

Item1,"[4], type = Float32, device = cpu 8 9 10 11"
Item2,"[2x4], type = Float32, device = cpu 4 5 6 7 8 9 10 11"


In [19]:
X[1, 2] = 9;
X

[3x4], type = Float32, device = cpu
 0 1 2 3
 4 5 9 7
 8 9 10 11

In [20]:
X[0..2, ..] = 12;
X

[3x4], type = Float32, device = cpu
 12 12 12 12
 12 12 12 12
 8 9 10 11

2.1.5. 节省内存

与 Python 不同， C# 不能单独自定义 `+=` 运算。这意味着以下两种做法在 TorchSharp 中不可能有任何区别：

In [21]:
var before = Y;
Y = Y + X;
object.ReferenceEquals(before, Y)

False

In [22]:
var before = Y;
Y += X;
object.ReferenceEquals(before, Y)

False

当然，切片是可以使用的：

In [23]:
var Z = torch.zeros_like(Y);
var before = Z;
Z[..] = X + Y;
object.ReferenceEquals(before, Z)

True

此外，与 PyTorch 不同，在 TorchSharp 中，我们需要额外地关心内存、显存的使用问题，其提供三种主要的[内存管理](https://github.com/dotnet/TorchSharp/wiki/Memory-Management)方式。

第一种方案是依赖 `Tensor` 类型的析构函数，其中会自动释放底层的 LibTorch 张量。这种方式胜在简单，即不用编写任何额外的代码。但由于析构函数是在垃圾回收时触发的，而在内存占用不高时，即使显存占用很高也不一定会触发垃圾回收，导致相关张量始终得不到释放，最终显存溢出。

第二种方案是手动调用张量的 `Dispose` 方法，或者使用 `using` 语句。这种方法看起来美好，但实际用起来非常繁琐。举例而言，仅仅是计算 `p = m + n + o;` ，为正确释放 `m + n` 的结果，必须改用 `using var temp = m + n; p = temp + o;` 。而这种情况常常大量出现在我们的代码中，因此这种方案并没有想象中那么漂亮。

因此我们有第三种方案，即 `torch.NewDisposeScope` 。在 `Tensor` 对象被创建时，会自动注册到其附近的 `DisposeScope` ，并随着 `DisposeScope` 的释放一同释放。通常我们会使用 `DisposeScope` 包裹一个小批量的训练过程，从而释放这一轮计算中得到的所有张量（因为下一轮开始就不会再用到它们了）。由于 `DisposeScope` 是 TorchSharp 额外提供的功能，或许我们需要通过一些例子来说明它：


In [24]:
var disposeScope = torch.NewDisposeScope();

var m = torch.zeros([]) + 1;
var n = torch.zeros([]) + 2;
var l = torch.zeros([]) + 3;
var p = m + n + l;
Console.WriteLine((double)p);

disposeScope.Dispose();

try
{
    Console.WriteLine((double)p);
}
catch (Exception ex)
{ 
    Console.WriteLine(ex.Message);
}

6
Tensor invalid -- empty handle.


当然，事实上我们会使用 `using` 而不是手动调用 `disposeScope.Dispose` 。

由于在 `DisposeScope` 中创建的张量都会随着它一同释放，因此当我们需要某个张量作为函数返回值的时候，可能需要把它移出函数内的 `DisposeScope` ，下面是一个相关的示例：

In [25]:
torch.Tensor MyTensorSum(torch.Tensor m, torch.Tensor n, torch.Tensor o)
{
    using (torch.NewDisposeScope())
    {
        var p = m + n + o;
        return p.MoveToOuterDisposeScope();
    }
    // 只有中间值 m + n 会在这里被释放。
    // m 、 n 、 o 都是在外界创建的，与此 DisposeScope 无关。
    // 而 p 已被移动到更外层的 DisposeScope （此处没有更外层，即最终它不关联任何 DisposeScope ）。
}

var m = torch.zeros([]) + 1;
var n = torch.zeros([]) + 2;
var l = torch.zeros([]) + 3;
var p = MyTensorSum(m, n, l);
Console.WriteLine((double)p);

6


有的时候，我们希望一个函数返回某个参数本身，但是外界可能误把原本的张量释放了（特别是当这个张量关联了某个 `DisposeScope` 时），导致返回值也同时被释放。为避免此问题，建议不要返回原本的对象，而是使用 `alias` 方法在 TorchSharp 层面创建一个新的 `Tensor` 对象。这两个张量在底层引用同一个 LibTorch 张量，只有两者都被释放后才会真正释放 LibTorch 张量。例如：

In [26]:
torch.Tensor MyFunction(torch.Tensor tensor)
{
    return tensor.alias();
}

torch.Tensor p;
using (var m = torch.zeros([]) + 123)
    p = MyFunction(m);
// 此时 m 已被释放，而 p 仍可用。
Console.WriteLine((double)p);

123


2.1.6. 转换为其他Python对象

将张量转换为 C# 数组是相对复杂的，因为要处理它的形状等问题。目前尚无直接转换为类似 `double[,]` 等数组的方案，但可以使用 `data` 获取张量元素的访问器，或者使用 `tolist` 转为由 `Tensor` 构成的 `ArrayList` 。这个 [Issue](https://github.com/dotnet/TorchSharp/issues/1287) 在讨论是否要加入更直接的方案。由于我们后面也并不会用到相关操作，此处就不提供相关内容了。

当然，从数组创建张量是简单的：

In [28]:
var B = torch.tensor(X.data<float>().ToArray());
(X, B)

Item1,"[3x4], type = Float32, device = cpu 12 12 12 12 12 12 12 12 8 9 10 11"
Item2,"[12], type = Float32, device = cpu 12 12 12 12 12 12 12 12 8 9 10 11"


可以使用 `item` 方法、强制转换，或者相关扩展方法转换标量：

In [34]:
using TorchSharp;

var a = torch.tensor(new double[] { 3.5 });
(a, a.item<double>(), (float)a, a.ToInt32())

Item1,"[1], type = Float64, device = cpu 3.5"
Item2,3.5
Item3,3.5
Item4,3


其与字符串的转换也值得一提，因为和 PyTorch 的表现不太一致。特别是，其默认使用 `TensorStringStyle.Metadata` ，不包含张量中的元素，可能带来误导。（例如你可能把一个标量误以为是空的张量，因为它只打印 `[], type = Float64, device = cpu` 。）更多相关信息可以参考[此处](https://github.com/dotnet/TorchSharp/wiki/Tensor-String-Formatting)。

In [35]:
Console.WriteLine($"ToString(): {a.ToString()}");
Console.WriteLine($"ToString(): {a.ToString(TensorStringStyle.Julia)}");
Console.WriteLine($"ToString(): {a.cstr()}");

ToString(): [1], type = Float64, device = cpu
ToString(): [1], type = Float64, device = cpu
 3.5

ToString(): [1], type = Float64, device = cpu, value = double [] {3.5}
